# Chapter 6. NLP
# Part 1. Tonality

#### - Data preparation (very slow)

In [1]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split

reviews_train = load_files("imdb/aclimdb/train")
reviews_test = load_files("imdb/aclimdb/train")

text_train, y_train = reviews_train.data, reviews_train.target
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

text_test, y_test = reviews_test.data, reviews_test.target
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

#### - Bag of words

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

bards_words = ["The fool doth think he is wise,", "but the wise man knows himself to be a fool"]
vect = CountVectorizer().fit(bards_words)
bag_of_words = vect.transform(bards_words)

print("Vocabulary size: {}".format( len( vect.vocabulary_)))
print("Vocabulary content: \n{}".format( vect.vocabulary_))
print("Sparse matrix content: \n{}".format( bag_of_words.toarray()))

Vocabulary size: 13
Vocabulary content: 
{'the': 9, 'fool': 3, 'doth': 2, 'think': 10, 'he': 4, 'is': 6, 'wise': 12, 'but': 1, 'man': 8, 'knows': 7, 'himself': 5, 'to': 11, 'be': 0}
Sparse matrix content: 
[[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


^ Sparse matrix shows how many tockens there is in document